# The `ParameterMapping`
In a heterogeneous `Dataset` that contains many `Replicate`s, one often needs to share some parameters globally across replicates while keeping others local, or shared to a subset of replicates.
With `murefi`, the rules for this mapping of parameters are defined in a `ParameterMapping`.

In [1]:
import numpy
import pandas

import murefi

## Creating a `ParameterMapping`
The `ParameterMapping` object is created from a `pandas.DataFrame`.
This dataframe must be indexed by the replicate ID (`"rid"`) and have the generic model parameter names as column headers.

The values in the `DataFrame` may be fixed to a specific value, or left variable by passing an arbitrary placeholder name.
Names of placeholders may occur multiple times to share them across respective replicates, but they must only appear within the same column.
In the following example, there are 6 free parameters ($substrate_c$, $P_0$, $v_{max,A/B/C}$ and $K_S$).
Here, only $K_S$ is truly *global*, whereas all other parameters apply to only a subset of the replicates.

In [2]:
df_mapping = pandas.DataFrame(
    columns='rid,S_0,P_0,v_max,K_S'.split(',')
).set_index('rid')
df_mapping.loc['A'] = (8.0, 'P_0', 'v_max_A', 'K_S')
df_mapping.loc['B'] = (10.0, 'P_0', 'v_max_B', 'K_S')
df_mapping.loc['C'] = ('substrate_c', '0', 'v_max_C', 'K_S')

df_mapping

,S_0,P_0,v_max,K_S
rid,,,,
A,8.0,P_0,v_max_A,K_S
B,10.0,P_0,v_max_B,K_S
C,substrate_c,0,v_max_C,K_S


The constructor of the `ParameterMapping` takes override-dictionaries for `bounds` and initial `guesses`.
These dictionaries should contain bounds & guesses for all model dimensions.

The entries in `bounds` & `guesses` *can* refer to the unique names of free parameters, but the more common use case is to use the same values for all parameters of the same kind:

In [3]:
# create the ParameterMapping object
pm = murefi.ParameterMapping(
    df_mapping,
    guesses=dict(
        S_0=5,
        P_0=0,
        v_max=2,
        K_S=0.1,
        # special guess for v_max_C:
        v_max_C=3,
    ),
    bounds=dict(
        S_0=(5, 15),
        P_0=(0, 5),
        v_max=(0, 10),
        K_S=(0.01, 1),
    )
)
pm

ParameterMapping(3 replicates, 4 inputs, 6 free parameters)

## Properties of the `ParameterMapping`
The object exposes several properties that are useful in various contexts.
This includes:
+ `order`: names of model parameters (matches the column order from the DataFrame)
+ `parameters`: dictionary of all unique parameters in the mapping and their corresponding model parameter name
+ `bounds` and `guesses` with `ndim` entries for all the `parameters`
+ `mapping` is a dictionary indicating the names or values of parameters that will end up in replicate-wise parameter vectors

In [4]:
print(f"""
Order of parameters in the model:   {pm.order}
Number of free parameters:          {pm.ndim}
Names of free parameters:           {pm.theta_names}

Initial guesses:    {pm.guesses}
Parameter bounds:   {pm.bounds}
""")


Order of parameters in the model:   ('S_0', 'P_0', 'v_max', 'K_S')
Number of free parameters:          6
Names of free parameters:           ('K_S', 'P_0', 'substrate_c', 'v_max_A', 'v_max_B', 'v_max_C')

Initial guesses:    (0.1, 0, 5, 2, 2, 3)
Parameter bounds:   ((0.01, 1), (0, 5), (5, 15), (0, 10), (0, 10), (0, 10))



The association between replicate ID and parameter values or placeholders is available through the `pm.mapping` property:

In [5]:
pm.mapping

{'A': (8.0, 'P_0', 'v_max_A', 'K_S'),
 'B': (10.0, 'P_0', 'v_max_B', 'K_S'),
 'C': ('substrate_c', 0.0, 'v_max_C', 'K_S')}

## The `repmap` Method
The `repmap` method is the most important feature - it maps a global parameter vector (same order as `theta_names`) to `Replicate`-wise parameter vectors.

For example, it can be used to map the full-length vector of initial guesses to short vectors for each replicate.
Note that for replicate "C", the 3rd parameter ($v_{max}$) has the 3, because it was given a special initial guess earlier.

In [6]:
pm.repmap(pm.guesses)

{'A': (8.0, 0, 2, 0.1), 'B': (10.0, 0, 2, 0.1), 'C': (5, 0.0, 3, 0.1)}

In [7]:
%load_ext watermark
%watermark -n -u -v -iv -w

Last updated: Mon Mar 29 2021

Python implementation: CPython
Python version       : 3.7.9
IPython version      : 7.19.0

murefi: 5.0.0
numpy : 1.19.2
pandas: 1.2.1

Watermark: 2.1.0

